In [1]:
import hyperspy.api as hs
# from PIL import Image
%matplotlib qt4
import matplotlib.pyplot as plt
import numpy as np
import cv2
import scipy as scp
import imreg_dft as ird
import os
from skimage import transform


fname_diff="/home/johannes/Desktop/Link to Data/AstartDataJulie/160512_SC239_X_overview_60k.blo"
fname_crop = "/home/johannes/Desktop/Link to Data/AstartDataJulie/diff-contact.blo"
fname_smallgrains = "/home/johannes/Desktop/Link to Data/160309JulieContact/SmallGrains_EDS Spectrum Image.dm3"
fname_nwcorner = "/home/johannes/Desktop/Link to Data/160309JulieContact/NWcorner_EDS Spectrum Image.dm3"
fname_nwcontact = "/home/johannes/Desktop/Link to Data/160309JulieContact/NWcontact_EDS Spectrum Image.dm3"
fname_contact='/media/johannes/149830F39830D4C8/Users/Johannes/OneDrive/Link to Documents/Materialfysikk/Data/160309JulieContact/contact_EDS Spectrum Image.dm3'

#USE HAADF, NOT BFTEM
fname_overview='/home/johannes/Desktop/Link to Data/160309JulieContact/BFTEM_overview.dm3'
fname_haadf1=   '/home/johannes/Desktop/Link to Data/160309JulieContact/HAADF_250k_3C.dm3'
fname_haadf2=   '/home/johannes/Desktop/Link to Data/160309JulieContact/HAADF_250k_8C.dm3'
fname_haadf3=   '/home/johannes/Desktop/Link to Data/160309JulieContact/HAADF_500k_8C.dm3'

unbcf_fast library is not present...
Falling back to slow python only backend.


In [ ]:
# sd=hs.load(fname_diff)
# overv=hs.load(fname_haadf1)
# overv2=hs.load(fname_overview)
# edx=hs.load(fname_nwcontact)
# edx.metadata
sd_sigcrop = hs.load('/home/johannes/Desktop/Link to Documents/Materialfysikk/PROJECT/Project/crops/num3/full_sd_sigcrop.hdf5')
edx=hs.load('/home/johannes/Desktop/Link to Documents/Materialfysikk/PROJECT/Project/machine_learning/edx.hdf')
# edx.plot()

In [17]:
sd_data=np.load('/home/johannes/Desktop/Link to Documents/Materialfysikk/PROJECT/Project/Oct9/sd_picture.npy')
se_data=np.load('/home/johannes/Desktop/Link to Documents/Materialfysikk/PROJECT/Project/Oct9/se_picture.npy')
sd_data_new=np.load('/home/johannes/Desktop/Link to Documents/Materialfysikk/PROJECT/Project/Oct9/sd_picture_uint8_rotated.npy')
se_data_sd=np.load('/home/johannes/Desktop/Link to Documents/Materialfysikk/PROJECT/Project/Oct9/se_picture_uint8.npy')

In [44]:
sd_data=sd_sigcrop.data.copy()
sd_data=sd_data.mean(axis=2).mean(axis=2)


se_data=edx.data.copy()
# se_data=sd_data.copy()
se_data=se_data.mean(axis=2)
# overv=hs.load(fname_haadf1)
# ov_data=overv.data

pts1 = np.float32([[103,133],[89,72]])
pts2 = np.float32([[50,53],[131,53]])
angle1=np.arctan2(pts1[0,1]-pts1[1,1],pts1[0,0]-pts1[1,0])
angle2=np.arctan2(pts2[0,1]-pts2[1,1],pts2[0,0]-pts2[1,0])
rot = angle1-angle2
scalex = (pts1[0,0]-pts1[1,0])/(pts2[0,0]-pts2[1,0])
scaley = (pts1[0,0]-pts1[1,0])/(pts2[0,0]-pts2[1,0])

shape1 = sd_data.shape
# shape2=ov_data.shape
offsetx = (pts2[0,0]-pts1[0,0])
print(scalex,scaley)
print(sd_data.shape),print(se_data.shape)#,print(ov_data.shape)
#sd: 60k, 200x260
#HAADF: 250k, 2048x2048
#EDX: 600k, 130x186
scale_sd_haadf = 60/250
scale_sd_edx=60/600
res_sd_edx=np.sqrt((200*200+260*260)/(130**2+186**2))#(2*2048*2048))
# print(res_sd_edx)
# print(scale_metadata)


plt.close('all')
print(rot)



# sd_data_new=transform.rotate(sd_data,np.rad2deg(rot))
sd_data_new=scp.ndimage.rotate(sd_data,angle=np.rad2deg(rot))
# sd_data_new=sd_data
print(res_sd_edx/scale_sd_edx)
plt.figure('rescaled_meta')
sd_data_new=(sd_data_new/sd_data_new.max()*255).astype('uint8')

plt.imshow(sd_data_new)

# plt.figure('overview')
# plt.imshow(overv.data)

plt.figure('se')
se_data_sd=se_data
se_data_sd=se_data_sd-se_data_sd.min()
se_data_sd=(se_data_sd/se_data_sd.max()*255).astype('uint8')

# ROTATE EDX INSTEAD:
# se_data_sd = transform.rotate(se_data_sd,-np.rad2deg(rot),resize=True)

plt.imshow(se_data_sd)

#plt.figure()
#plt.imshow(scp.ndimage.affine_transform(sd_sigcrop.mean(axis=2).mean(axis=2)))
#sd_sigcrop.plot()
print(angle1,angle2)

NameError: name 'sd_sigcrop' is not defined

In [33]:
import cv2
plt.close('all')
sd_data_new = sd_data_rescale_rot
_im2=(sd_data_new/sd_data_new.max()*255).astype('uint8')
# _im2[100:,:]=0
im1=se_data_sd-se_data_sd.min()
im1=(im1/im1.max()*255).astype('uint8')
im1[im1<50]=0
im1=im1[11:,:]

# im1 = edx.data.mean(axis=2)
# im1=(im1/im1.max()*255).astype('uint8')
# _im2 = (overv.data/overv.data.max()*255).astype('uint8')

plt.figure(),plt.imshow(im1)
plt.figure(),plt.imshow(_im2)

w, h = im1.shape[::-1]
print(im1.shape)
methods = np.array(['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR','cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED'])
for m in methods:
    if m in ['cv2.TM_CCOEFF','cv2.TM_CCORR']:
        continue
    im2=_im2.copy()
    plt.figure(m)
    method=eval(m)
    res = cv2.matchTemplate(im2, im1, method)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
        top_left = min_loc
    else:
        top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)

    cv2.rectangle(im2,top_left, bottom_right, 255, 1)

    plt.subplot(121),plt.imshow(res,cmap = 'gray')
    plt.title('Matching Result'), plt.xticks([]), plt.yticks([])
    plt.subplot(122),plt.imshow(im2,cmap = 'gray')
    plt.title('Detected Point'), plt.xticks([]), plt.yticks([])
    plt.suptitle(m)
    plt.hold('off')
    plt.show()

(119, 186)


## Find scale

In [13]:
sd_shape=sd_data.shape
se_shape=se_data.shape
sd_zoom=60
se_zoom=600
plt.figure(),plt.imshow(sd_data)
plt.figure(),plt.imshow(se_data)

(<matplotlib.figure.Figure at 0x7f96a76332e8>,
 <matplotlib.image.AxesImage at 0x7f96a3b5a8d0>)

In [6]:
plt.close('all')
print(top_left, bottom_right)
h,w=sd_data.shape
sd_data_crop=sd_data_new[top_left[1]:bottom_right[1],top_left[0]:bottom_right[0]]
plt.figure()
plt.imshow(sd_data_crop)
plt.figure()
plt.imshow(se_data_sd)

NameError: name 'top_left' is not defined

In [12]:
# ANGLE:
angle = np.rad2deg(rot)
print('Angle:',angle)
print('x:',(top_left[1],bottom_right[1]))
print('y:',(top_left[0],bottom_right[0]))

Angle: -102.926
x: (7, 137)
y: (45, 231)


In [ ]:
### scale_sd_haadf = 60/250
# scale_sd_edx=60/600
# res_sd_edx=np.sqrt((200*200+260*260)/(2*2048*2048))
# edx=hs.load(fname_nwcorner)
# edx.map(scp.ndimage.interpolation.rotate, angle=5, reshape=True)
# plt.clf()
# plt.cla()
# plt.close('all')
# edx.plot()
sd=sd.T
sd.map(scp.ndimage.interpolation.rotate, angle=+102.926, axes=(0,1), reshape=True)
sd.crop_image(top=45,bottom=231,left=7,right=137)
sd=sd.T
sd.plot()

In [ ]:
sd.save("/home/johannes/Desktop/Link to Data/AstartDataJulie/160512_SC239_X_overview_60k.hdf5")

In [7]:
plt.imshow(sd.data.mean(axis=(2,3)))

In [6]:
fname_nwcontact = "/home/johannes/Desktop/Link to Data/160309JulieContact/NWcontact_EDS Spectrum Image.dm3"
edx=hs.load(fname_nwcontact)
edx

<EDSTEMSpectrum, title: NWcontact_EDS Spectrum Image, dimensions: (186, 130|2048)>

## Find scalefactor, rotate, ImReg

In [10]:
fname_diff="/home/johannes/Desktop/Link to Data/AstartDataJulie/160512_SC239_X_overview_60k.blo"

# sd = hs.load(fname_diff)
edx=hs.load(fname_nwcontact)

# sd.plot()

In [11]:
edx.axes_manager

<Axes manager, axes: (186, 130|2048)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
               x |    186 |      0 |      -0 |  0.0013 |     µm 
               y |    130 |      0 |      -0 |  0.0013 |     µm 
---------------- | ------ | ------ | ------- | ------- | ------ 
               z |   2048 |        |      -0 |    0.01 |    keV

In [43]:
# edx_scale = edx.axes_manager.as_dictionary()['axis-1']['scale']*1000
edx_scale = 0.0013385306810960174*1000
# sd_scale = sd.axes_manager.as_dictionary()['axis-1']['scale']
sd_scale = 1.92
print(edx_scale)
print(sd_scale)
# plt.imshow(sd_data)
# sd_data_rescale=transform.rescale(sd_data, sd_scale/edx_scale)
se_data = transform.rescale(se_data_sd, edx_scale/sd_scale)
sd_data_rescale = sd_data
sd_data_rescale_rot=scp.ndimage.rotate(sd_data_rescale, -102.926)

1.3385306810960174
1.92


NameError: name 'se_data_sd' is not defined

In [19]:
plt.close('all')
plt.imshow(sd_data_rescale_rot)
plt.figure()
plt.imshow(sd_data)
plt.figure()
plt.imshow(se_data)

### ImReg

In [42]:
# import cv2
plt.close('all')
sd_data_new = sd_data_rescale_rot
se_data_sd = se_data
_im2=(sd_data_new/sd_data_new.max()*255).astype('uint8')
# _im2[100:,:]=0
im1=se_data_sd-se_data_sd.min()
im1=(im1/im1.max()*255).astype('uint8')
# im1[im1<50]=0
# im1=im1[11:,:]

# im1 = edx.data.mean(axis=2)
# im1=(im1/im1.max()*255).astype('uint8')
# _im2 = (overv.data/overv.data.max()*255).astype('uint8')

plt.figure(),plt.imshow(im1)
plt.figure(),plt.imshow(_im2)

w, h = im1.shape[::-1]
print(im1.shape)
methods = np.array(['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR','cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED'])
m = 'cv2.TM_SQDIFF_NORMED'
im2=_im2.copy()
plt.figure(m)
method=eval(m)
res = cv2.matchTemplate(im2, im1, method)
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
    top_left = min_loc
else:
    top_left = max_loc
bottom_right = (top_left[0] + w, top_left[1] + h)

cv2.rectangle(im2,top_left, bottom_right, 255, 1)

plt.subplot(121),plt.imshow(res,cmap = 'gray')
plt.title('Matching Result'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(im2,cmap = 'gray')
plt.title('Detected Point'), plt.xticks([]), plt.yticks([])
plt.suptitle(m)
plt.hold('off')
plt.show()

NameError: name 'sd_data_rescale_rot' is not defined

In [21]:
print(top_left[1], bottom_right[1],top_left[0], bottom_right[0])

76 167 67 197


### Cut piece out

In [22]:
plt.close('all')
print(top_left, bottom_right)
sd_data_crop=sd_data_new[top_left[1]:bottom_right[1],top_left[0]:bottom_right[0]]

plt.figure()
plt.imshow(sd_data_crop)
plt.figure()
plt.imshow(se_data_sd)

(67, 76) (197, 167)


## Rotate SPED

In [2]:
# sd=hs.load(fname_diff)
sd = hs.load('/home/johannes/Desktop/Link to Documents/Materialfysikk/PROJECT/Project/crops/num3/full_sd_sigcrop.hdf5')
sd.plot()

In [28]:
s=sd.rotate(angle=-102.93, reshape = True,rotate_dimension='navigation')
# s=s.inav[110:240,97:283]
s.plot()

In [5]:
s_cut=s.inav[67:197,76:167]
s_cut.plot()

In [23]:
# edx=hs.load(fname_nwcontact)
edx.plot()

In [34]:
edx_scale = 0.0013385306810960174*1000
sd_scale = 1.92

In [41]:
for i in range(2):
    edx.axes_manager.navigation_axes[i].index = int(s_cut.axes_manager.navigation_axes[i].index*sd_scale/edx_scale)